In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [11]:
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')

In [12]:
movies.drop(['genres'], axis=1, inplace=True)
movies.head(3)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)


In [13]:
ratings.drop(['timestamp'], axis=1, inplace=True)
ratings.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


In [15]:
user_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')
user_item_matrix.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


In [17]:
user_item_matrix.fillna(0, inplace=True)
user_item_matrix.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [31]:
users_votes = ratings.groupby('userId')['rating'].agg('count')
movies_votes = ratings.groupby('movieId')['rating'].agg('count')

user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index

user_item_matrix = user_item_matrix.loc[movie_mask,:]
user_item_matrix = user_item_matrix.loc[:,user_mask]

In [32]:
user_item_matrix.shape

(2121, 378)

In [34]:
csr_data = csr_matrix(user_item_matrix.values)

In [35]:
print(csr_data[:2, :5])

  (0, 0)	4.0
  (0, 3)	4.5
  (1, 2)	4.0


In [36]:
user_item_matrix = user_item_matrix.rename_axis(None, axis=1).reset_index()
user_item_matrix.head(3)

,movieId,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [38]:
knn = NearestNeighbors(metric='cosine',
                      algorithm='brute',
                      n_neighbors=20,
                      n_jobs=-1)

knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [50]:
recommendations = 10
search_word = 'Matrix'

In [51]:
movie_search = movies[movies['title'].str.contains(search_word)]
movie_search

,movieId,title
1939,2571,"Matrix, The (1999)"
4351,6365,"Matrix Reloaded, The (2003)"
4639,6934,"Matrix Revolutions, The (2003)"


In [52]:
movie_id = movie_search.iloc[0]['movieId']
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]

movie_id

901

In [54]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors=recommendations + 1)

In [60]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list, distances_list))

print(indices_distances[:3])

[(901, 0.0), (1002, 0.22982440568634488), (442, 0.25401128310081567)]


In [63]:
indices_distances_sorted = sorted(indices_distances, key=lambda x: x[1])[1:]
indices_distances_sorted

[(1002, 0.22982440568634488),
 (442, 0.25401128310081567),
 (454, 0.27565616686043737),
 (124, 0.2776088577731709),
 (735, 0.2869100842838125),
 (954, 0.2911101181714415),
 (1362, 0.31393358217709477),
 (1157, 0.31405925934381695),
 (1536, 0.3154800434449465),
 (978, 0.31748544046311844)]

In [75]:
recommendations = []

for index, distance in indices_distances_sorted:
    matrix_movie_id = user_item_matrix.iloc[index]['movieId']
    
    id = movies[movies['movieId'] == matrix_movie_id].index
    title = movies.iloc[id]['title'].values[0]
    
    recommendations.append({'Title': title, 'Distance': distance})

In [76]:
recommendations[0]

{'Title': 'Fight Club (1999)', 'Distance': 0.22982440568634488}

In [78]:
recommendations_df = pd.DataFrame(recommendations, index=range(1, len(recommendations) + 1))
recommendations_df

,Title,Distance
1,Fight Club (1999),0.229824
2,Star Wars: Episode V - The Empire Strikes Back...,0.254011
3,Star Wars: Episode VI - Return of the Jedi (1983),0.275656
4,Star Wars: Episode IV - A New Hope (1977),0.277609
5,Saving Private Ryan (1998),0.286910
6,"Sixth Sense, The (1999)",0.291110
7,"Lord of the Rings: The Fellowship of the Ring,...",0.313934
8,Gladiator (2000),0.314059
9,"Lord of the Rings: The Return of the King, The...",0.315480
10,American Beauty (1999),0.317485
